---

# 🔧 PART 1: Environment Setup

Load all necessary components for simulation and ML evaluation.

In [4]:
# Install required packages
%pip install simpy --quiet

Note: you may need to restart the kernel to use updated packages.


In [7]:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Add project paths - use absolute paths for THIS specific project
project_root = Path.cwd().parent if 'notebook' in str(Path.cwd()) else Path.cwd()
if 'can_stadium_AI' not in str(project_root):
    # Navigate to the correct project root
    project_root = Path(r'c:\Users\Woundex\Documents\Project\can_stadium_AI')

sys.path.insert(0, str(project_root))
sys.path.insert(0, str(project_root / 'simulation'))
sys.path.insert(0, str(project_root / 'ml'))

# Change to project root to help with relative imports in simulation modules
os.chdir(project_root)

# Plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print('✅ Paths configured')
print(f'📁 Project root: {project_root}')
print(f'📁 Working directory: {os.getcwd()}')

✅ Paths configured
📁 Project root: c:\Users\Woundex\Documents\Project\can_stadium_AI
📁 Working directory: c:\Users\Woundex\Documents\Project\can_stadium_AI


## Load ML Models

We need trained models to drive intelligent control decisions.

In [8]:
# Import simulation components
try:
    from run_simulation import run_match_simulation
    from resources import StadiumResources
    from metrics import MetricsCollector
    print('✅ Simulation components imported')
except ImportError as e:
    print(f'⚠️  Simulation import error: {e}')
    raise

# Load trained ML models
try:
    from load_models import load_all_models
    models = load_all_models()
    print('✅ ML Models Loaded:')
    for name, model in models.items():
        if model is not None:
            print(f"   - {name.title()} Model: {type(model).__name__}")
        else:
            print(f"   - {name.title()} Model: Not available")
    ML_AVAILABLE = any(m is not None for m in models.values())
    if not ML_AVAILABLE:
        print('\n⚠️  No trained models found - will run baseline-only comparison')
        print('   Train models in notebook 01_explore_prepare.ipynb first')
except Exception as e:
    print(f'⚠️  ML models not available: {e}')
    print('   Will run baseline-only comparison')
    models = None
    ML_AVAILABLE = False

✅ Simulation components imported
✅ ML Models Loaded:
   - Crowd Model: Not available
   - Queue Model: Not available
   - Anomaly Model: Not available

⚠️  No trained models found - will run baseline-only comparison
   Train models in notebook 01_explore_prepare.ipynb first


---

# 🎮 PART 2: Simulation Configuration

Define the exact parameters for both baseline and ML scenarios.

In [10]:
# Simulation parameters
NUM_REPLICATIONS = 3  # Reduced for faster testing (use 10 for full evaluation)
RANDOM_SEED_BASE = 42  # Reproducibility

# Stadium configuration (common to both scenarios)
COMMON_CONFIG = {
    'total_fans': 68000,
    'capacity': 72000,
    'num_turnstiles': 92,
    'parking_spots': 6000,
    'simulation_duration': 400,  # minutes (covers full match + exit)
}

# Baseline configuration (no control)
BASELINE_CONFIG = {
    **COMMON_CONFIG,
    'num_vendors': 80,  # Fixed vendor count
    'turnstile_service_factor': 1.0,  # Normal speed
    'redirection_enabled': False,  # No intelligent routing
    'control_policy': 'none',
}

# ML-controlled configuration
ML_CONFIG = {
    **COMMON_CONFIG,
    'num_vendors': 80,  # Base vendor count (will adjust dynamically)
    'turnstile_service_factor': 1.0,  # Base speed (will adjust dynamically)
    'redirection_enabled': True,  # Intelligent routing
    'control_policy': 'ml_driven',
    'intervention_threshold': 10,  # Act when queue > 10 min wait
    'vendor_adjustment_step': 5,  # Add vendors in groups of 5
    'throttle_factor_range': (0.8, 1.2),  # ±20% service rate adjustment
}

print('📋 Configuration Summary:')
print(f'   Replications per scenario: {NUM_REPLICATIONS}')
print(f'   Total fans per match: {COMMON_CONFIG["total_fans"]:,}')
print(f'   Simulation duration: {COMMON_CONFIG["simulation_duration"]} minutes')
print(f'\n🔹 Baseline: Static operations')
print(f'🔹 ML-Controlled: Dynamic adjustment every 5 minutes')

📋 Configuration Summary:
   Replications per scenario: 3
   Total fans per match: 68,000
   Simulation duration: 400 minutes

🔹 Baseline: Static operations
🔹 ML-Controlled: Dynamic adjustment every 5 minutes


---

# 🏃 PART 3: Run Baseline Simulations

Execute multiple baseline runs to establish the **reference performance**.

In [11]:
print('🎬 Starting BASELINE simulations...')
print('=' * 60)

baseline_results = []

for rep in range(NUM_REPLICATIONS):
    seed = RANDOM_SEED_BASE + rep
    print(f'\n🔄 Baseline Run {rep+1}/{NUM_REPLICATIONS} (seed={seed})')
    
    # Run simulation
    metrics = run_match_simulation(
        num_fans=BASELINE_CONFIG['total_fans'],
        num_turnstiles=BASELINE_CONFIG['num_turnstiles'],
        num_vendors=BASELINE_CONFIG['num_vendors'],
        parking_spots=BASELINE_CONFIG['parking_spots'],
        random_seed=seed,
        enable_ml_control=False  # KEY: No ML intervention
    )
    
    # Store results
    baseline_results.append({
        'replication': rep + 1,
        'seed': seed,
        'metrics': metrics,
        'scenario': 'baseline'
    })
    
    # Quick summary
    if metrics.turnstile_waits:
        avg_wait = np.mean(metrics.turnstile_waits)
        max_wait = np.max(metrics.turnstile_waits)
        print(f'   ├─ Avg turnstile wait: {avg_wait:.2f} min')
        print(f'   ├─ Max turnstile wait: {max_wait:.2f} min')
        print(f'   └─ Fans completed: {metrics.fans_completed:,}')

print('\n' + '=' * 60)
print(f'✅ Baseline simulations complete: {len(baseline_results)} runs')

🎬 Starting BASELINE simulations...

🔄 Baseline Run 1/3 (seed=42)
t=   10 | In Stadium:    612 | Exited:      0 | Queue:     0 | Arr:  108/min | Exit:    0/min
t=   20 | In Stadium:   1615 | Exited:      0 | Queue:     0 | Arr:  103/min | Exit:    0/min
t=   30 | In Stadium:   2622 | Exited:      0 | Queue:     0 | Arr:  105/min | Exit:    0/min
t=   40 | In Stadium:   3616 | Exited:      0 | Queue:     0 | Arr:   78/min | Exit:    0/min
t=   50 | In Stadium:   4584 | Exited:      0 | Queue:     0 | Arr:  101/min | Exit:    0/min
t=   60 | In Stadium:   5588 | Exited:      0 | Queue:     0 | Arr:   88/min | Exit:    0/min
t=   70 | In Stadium:   8960 | Exited:      0 | Queue:     0 | Arr:  491/min | Exit:    0/min
t=   80 | In Stadium:  13949 | Exited:      0 | Queue:     0 | Arr:  478/min | Exit:    0/min
t=   90 | In Stadium:  19290 | Exited:      0 | Queue:     0 | Arr:  474/min | Exit:    0/min
t=  100 | In Stadium:  27735 | Exited:      0 | Queue:     0 | Arr:  997/min | Exit:    0

## Baseline Performance Summary

In [ ]:
# Aggregate baseline metrics
baseline_stats = {
    'avg_turnstile_wait': [],
    'max_turnstile_wait': [],
    'avg_vendor_wait': [],
    'peak_queue_length': [],
    'avg_exit_wait': [],
    'fans_completed': [],
}

for result in baseline_results:
    m = result['metrics']
    if m.turnstile_waits:
        baseline_stats['avg_turnstile_wait'].append(np.mean(m.turnstile_waits))
        baseline_stats['max_turnstile_wait'].append(np.max(m.turnstile_waits))
    if m.vendor_waits:
        baseline_stats['avg_vendor_wait'].append(np.mean(m.vendor_waits))
    if hasattr(m, 'queue_lengths') and m.queue_lengths:
        baseline_stats['peak_queue_length'].append(max(m.queue_lengths))
    if hasattr(m, 'exit_waits') and m.exit_waits:
        baseline_stats['avg_exit_wait'].append(np.mean(m.exit_waits))
    baseline_stats['fans_completed'].append(m.fans_completed)

# Create summary DataFrame
baseline_summary = pd.DataFrame({
    'Metric': list(baseline_stats.keys()),
    'Mean': [np.mean(v) if v else 0 for v in baseline_stats.values()],
    'Std': [np.std(v) if v else 0 for v in baseline_stats.values()],
    'Min': [np.min(v) if v else 0 for v in baseline_stats.values()],
    'Max': [np.max(v) if v else 0 for v in baseline_stats.values()],
})

print('\n📊 BASELINE PERFORMANCE SUMMARY')
print('=' * 80)
print(baseline_summary.to_string(index=False))
print('=' * 80)


📊 BASELINE PERFORMANCE SUMMARY
            Metric         Mean      Std          Min          Max
avg_turnstile_wait     0.000000 0.000000     0.000000     0.000000
max_turnstile_wait     0.000000 0.000000     0.000000     0.000000
   avg_vendor_wait     0.095669 0.049601     0.042845     0.162051
 peak_queue_length     0.000000 0.000000     0.000000     0.000000
     avg_exit_wait     0.000000 0.000000     0.000000     0.000000
    fans_completed 68000.000000 0.000000 68000.000000 68000.000000


: 

---

# 🤖 PART 4: Run ML-Controlled Simulations

Now execute simulations with **intelligent control policies** driven by ML predictions.

In [ ]:
if not ML_AVAILABLE:
    print('⚠️  Skipping ML simulations - models not available')
    print('   Train models first using notebook 01_explore_prepare.ipynb')
else:
    print('🎬 Starting ML-CONTROLLED simulations...')
    print('=' * 60)
    
    ml_results = []
    
    for rep in range(NUM_REPLICATIONS):
        seed = RANDOM_SEED_BASE + rep  # Same seeds for fair comparison
        print(f'\n🔄 ML-Controlled Run {rep+1}/{NUM_REPLICATIONS} (seed={seed})')
        
        # Run simulation with ML control
        metrics = run_match_simulation(
            num_fans=ML_CONFIG['total_fans'],
            num_turnstiles=ML_CONFIG['num_turnstiles'],
            num_vendors=ML_CONFIG['num_vendors'],
            parking_spots=ML_CONFIG['parking_spots'],
            random_seed=seed,
            enable_ml_control=True,  # KEY: Enable ML intervention
            ml_models=models,
            intervention_threshold=ML_CONFIG['intervention_threshold'],
        )
        
        # Store results
        ml_results.append({
            'replication': rep + 1,
            'seed': seed,
            'metrics': metrics,
            'scenario': 'ml_controlled'
        })
        
        # Quick summary
        if metrics.turnstile_waits:
            avg_wait = np.mean(metrics.turnstile_waits)
            max_wait = np.max(metrics.turnstile_waits)
            print(f'   ├─ Avg turnstile wait: {avg_wait:.2f} min')
            print(f'   ├─ Max turnstile wait: {max_wait:.2f} min')
            print(f'   └─ Fans completed: {metrics.fans_completed:,}')
    
    print('\n' + '=' * 60)
    print(f'✅ ML-controlled simulations complete: {len(ml_results)} runs')

## ML-Controlled Performance Summary

In [ ]:
if ML_AVAILABLE:
    # Aggregate ML metrics
    ml_stats = {
        'avg_turnstile_wait': [],
        'max_turnstile_wait': [],
        'avg_vendor_wait': [],
        'peak_queue_length': [],
        'avg_exit_wait': [],
        'fans_completed': [],
    }
    
    for result in ml_results:
        m = result['metrics']
        if m.turnstile_waits:
            ml_stats['avg_turnstile_wait'].append(np.mean(m.turnstile_waits))
            ml_stats['max_turnstile_wait'].append(np.max(m.turnstile_waits))
        if m.vendor_waits:
            ml_stats['avg_vendor_wait'].append(np.mean(m.vendor_waits))
        if hasattr(m, 'queue_lengths') and m.queue_lengths:
            ml_stats['peak_queue_length'].append(max(m.queue_lengths))
        if hasattr(m, 'exit_waits') and m.exit_waits:
            ml_stats['avg_exit_wait'].append(np.mean(m.exit_waits))
        ml_stats['fans_completed'].append(m.fans_completed)
    
    # Create summary DataFrame
    ml_summary = pd.DataFrame({
        'Metric': list(ml_stats.keys()),
        'Mean': [np.mean(v) if v else 0 for v in ml_stats.values()],
        'Std': [np.std(v) if v else 0 for v in ml_stats.values()],
        'Min': [np.min(v) if v else 0 for v in ml_stats.values()],
        'Max': [np.max(v) if v else 0 for v in ml_stats.values()],
    })
    
    print('\n📊 ML-CONTROLLED PERFORMANCE SUMMARY')
    print('=' * 80)
    print(ml_summary.to_string(index=False))
    print('=' * 80)

---

# 📈 PART 5: Statistical Comparison

**The moment of truth**: Is ML significantly better than baseline?

In [ ]:
if ML_AVAILABLE:
    print('\n🔬 STATISTICAL SIGNIFICANCE TESTING')
    print('=' * 80)
    print('Using paired t-tests (α = 0.05)\n')
    
    comparison_results = []
    
    for metric_name in baseline_stats.keys():
        baseline_values = np.array(baseline_stats[metric_name])
        ml_values = np.array(ml_stats[metric_name])
        
        if len(baseline_values) == 0 or len(ml_values) == 0:
            continue
        
        # Compute improvement
        baseline_mean = np.mean(baseline_values)
        ml_mean = np.mean(ml_values)
        improvement = baseline_mean - ml_mean
        improvement_pct = (improvement / baseline_mean * 100) if baseline_mean > 0 else 0
        
        # Statistical test
        if len(baseline_values) == len(ml_values):
            t_stat, p_value = stats.ttest_rel(baseline_values, ml_values)
        else:
            t_stat, p_value = stats.ttest_ind(baseline_values, ml_values)
        
        significant = '✅ YES' if p_value < 0.05 else '❌ NO'
        
        comparison_results.append({
            'Metric': metric_name,
            'Baseline': f'{baseline_mean:.2f}',
            'ML': f'{ml_mean:.2f}',
            'Δ': f'{improvement:.2f}',
            'Δ%': f'{improvement_pct:.1f}%',
            'p-value': f'{p_value:.4f}',
            'Significant': significant
        })
        
        print(f'📊 {metric_name}')
        print(f'   Baseline: {baseline_mean:.2f} → ML: {ml_mean:.2f}')
        print(f'   Improvement: {improvement:.2f} ({improvement_pct:.1f}%)')
        print(f'   p-value: {p_value:.4f} → {significant}')
        print()
    
    comparison_df = pd.DataFrame(comparison_results)
    print('\n' + '=' * 80)
    print('📋 COMPARISON SUMMARY')
    print('=' * 80)
    print(comparison_df.to_string(index=False))
    print('=' * 80)

---

# 📊 PART 6: Visualization - Side-by-Side Comparison

Visual proof of ML impact.

In [ ]:
if ML_AVAILABLE:
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('🏟️ Baseline vs ML-Controlled Stadium Performance', 
                 fontsize=18, fontweight='bold', y=1.00)
    
    # Plot 1: Turnstile Wait Times
    ax1 = axes[0, 0]
    data1 = pd.DataFrame({
        'Baseline': baseline_stats['avg_turnstile_wait'],
        'ML-Controlled': ml_stats['avg_turnstile_wait']
    })
    data1.plot(kind='box', ax=ax1, color={'boxes': ['coral', 'lightblue']})
    ax1.set_title('Average Turnstile Wait Time', fontsize=14, fontweight='bold')
    ax1.set_ylabel('Wait Time (minutes)', fontsize=12)
    ax1.grid(axis='y', alpha=0.3)
    
    # Plot 2: Vendor Wait Times
    ax2 = axes[0, 1]
    if baseline_stats['avg_vendor_wait'] and ml_stats['avg_vendor_wait']:
        data2 = pd.DataFrame({
            'Baseline': baseline_stats['avg_vendor_wait'],
            'ML-Controlled': ml_stats['avg_vendor_wait']
        })
        data2.plot(kind='box', ax=ax2, color={'boxes': ['coral', 'lightblue']})
    ax2.set_title('Average Vendor Wait Time', fontsize=14, fontweight='bold')
    ax2.set_ylabel('Wait Time (minutes)', fontsize=12)
    ax2.grid(axis='y', alpha=0.3)
    
    # Plot 3: Peak Queue Length
    ax3 = axes[1, 0]
    if baseline_stats['peak_queue_length'] and ml_stats['peak_queue_length']:
        data3 = pd.DataFrame({
            'Baseline': baseline_stats['peak_queue_length'],
            'ML-Controlled': ml_stats['peak_queue_length']
        })
        data3.plot(kind='box', ax=ax3, color={'boxes': ['coral', 'lightblue']})
    ax3.set_title('Peak Queue Length', fontsize=14, fontweight='bold')
    ax3.set_ylabel('Number of Fans', fontsize=12)
    ax3.grid(axis='y', alpha=0.3)
    
    # Plot 4: Improvement Summary Bar Chart
    ax4 = axes[1, 1]
    if comparison_results:
        metrics_to_plot = [r for r in comparison_results if r['Metric'] != 'fans_completed']
        metric_names = [r['Metric'].replace('_', ' ').title() for r in metrics_to_plot]
        improvements = [float(r['Δ%'].strip('%')) for r in metrics_to_plot]
        
        colors = ['green' if x > 0 else 'red' for x in improvements]
        ax4.barh(metric_names, improvements, color=colors, alpha=0.7)
        ax4.axvline(0, color='black', linewidth=0.8, linestyle='--')
        ax4.set_xlabel('Improvement (%)', fontsize=12)
        ax4.set_title('ML Impact: Percentage Improvement', fontsize=14, fontweight='bold')
        ax4.grid(axis='x', alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('../data/processed/baseline_vs_ml_comparison.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print('\n✅ Comparison plots saved to: data/processed/baseline_vs_ml_comparison.png')

---

# 🎯 PART 7: Policy Impact Analysis

Which control actions drove the most value?

In [ ]:
if ML_AVAILABLE:
    print('\n🎛️ CONTROL POLICY IMPACT ANALYSIS')
    print('=' * 80)
    
    # Analyze policy actions from ML runs
    total_interventions = 0
    vendor_adjustments = 0
    throttle_adjustments = 0
    redirection_uses = 0
    
    for result in ml_results:
        m = result['metrics']
        # Count interventions from metrics
        if hasattr(m, 'control_actions'):
            total_interventions += len(m.control_actions)
            for action in m.control_actions:
                if 'vendor' in action.lower():
                    vendor_adjustments += 1
                if 'throttle' in action.lower():
                    throttle_adjustments += 1
                if 'redirect' in action.lower():
                    redirection_uses += 1
    
    print(f'Total ML Interventions: {total_interventions}')
    print(f'├─ Vendor Adjustments: {vendor_adjustments}')
    print(f'├─ Turnstile Throttling: {throttle_adjustments}')
    print(f'└─ Redirection Actions: {redirection_uses}')
    print()
    
    # Compute ROI
    if baseline_stats['avg_turnstile_wait'] and ml_stats['avg_turnstile_wait']:
        baseline_avg = np.mean(baseline_stats['avg_turnstile_wait'])
        ml_avg = np.mean(ml_stats['avg_turnstile_wait'])
        time_saved_per_fan = baseline_avg - ml_avg  # minutes
        total_fans = COMMON_CONFIG['total_fans']
        total_time_saved = time_saved_per_fan * total_fans / 60  # hours
        
        print('💰 BUSINESS VALUE CALCULATION')
        print('=' * 80)
        print(f'Time saved per fan: {time_saved_per_fan:.2f} minutes')
        print(f'Total time saved (68,000 fans): {total_time_saved:.0f} hours')
        print(f'Equivalent to: {total_time_saved/8:.0f} person-days of productivity')
        print()
        print('📊 Fan Satisfaction Impact:')
        satisfaction_improvement = (
            100 * (baseline_avg - ml_avg) / baseline_avg
        )
        print(f'   Wait time reduction: {satisfaction_improvement:.1f}%')
        print(f'   Expected satisfaction gain: +{satisfaction_improvement/3:.1f}%')
        print('=' * 80)

---

# 📝 PART 8: Executive Summary Report

Generate a concise report for stakeholders.

In [ ]:
if ML_AVAILABLE:
    report = []
    report.append('=' * 80)
    report.append('🏟️  STADIUM ML CONTROL SYSTEM: EVALUATION REPORT')
    report.append('=' * 80)
    report.append('')
    report.append('## EXECUTIVE SUMMARY')
    report.append('')
    report.append(f'Evaluation Date: {pd.Timestamp.now().strftime("%Y-%m-%d")}')
    report.append(f'Scenarios Tested: {NUM_REPLICATIONS} replications each')
    report.append(f'Stadium Capacity: {COMMON_CONFIG["capacity"]:,} fans')
    report.append(f'Match Attendance: {COMMON_CONFIG["total_fans"]:,} fans')
    report.append('')
    report.append('## KEY FINDINGS')
    report.append('')
    
    for result in comparison_results[:4]:  # Top 4 metrics
        metric = result['Metric'].replace('_', ' ').title()
        improvement = result['Δ%']
        sig = '(statistically significant)' if 'YES' in result['Significant'] else '(not significant)'
        report.append(f'✓ {metric}: {improvement} improvement {sig}')
    
    report.append('')
    report.append('## RECOMMENDATIONS')
    report.append('')
    report.append('1. Deploy ML-controlled system for high-capacity matches (>65,000 fans)')
    report.append('2. Implement predictive staffing 60 minutes before kickoff')
    report.append('3. Enable dynamic turnstile throttling during peak entry (t=150-180 min)')
    report.append('4. Monitor queue predictions and adjust vendor capacity in real-time')
    report.append('5. Collect continuous feedback for model retraining')
    report.append('')
    report.append('## CONCLUSION')
    report.append('')
    if any('YES' in r['Significant'] for r in comparison_results):
        report.append('✅ ML-driven control policies demonstrate STATISTICALLY SIGNIFICANT')
        report.append('   improvements in stadium operations. Deployment recommended.')
    else:
        report.append('⚠️  ML improvements detected but not statistically significant.')
        report.append('   Additional training data or model refinement recommended.')
    
    report.append('')
    report.append('=' * 80)
    
    report_text = '\n'.join(report)
    print(report_text)
    
    # Save report
    report_path = Path('../data/processed/ml_evaluation_report.txt')
    report_path.write_text(report_text)
    print(f'\n✅ Report saved to: {report_path}')

---

# 🎓 PART 9: Export Results for Publication

Prepare data for academic papers, presentations, or dashboards.

In [ ]:
if ML_AVAILABLE:
    # Create comprehensive results dataset
    export_data = []
    
    # Baseline data
    for result in baseline_results:
        m = result['metrics']
        export_data.append({
            'scenario': 'baseline',
            'replication': result['replication'],
            'avg_turnstile_wait': np.mean(m.turnstile_waits) if m.turnstile_waits else 0,
            'max_turnstile_wait': np.max(m.turnstile_waits) if m.turnstile_waits else 0,
            'avg_vendor_wait': np.mean(m.vendor_waits) if m.vendor_waits else 0,
            'fans_completed': m.fans_completed,
            'fans_exited': getattr(m, 'fans_exited', 0),
        })
    
    # ML data
    for result in ml_results:
        m = result['metrics']
        export_data.append({
            'scenario': 'ml_controlled',
            'replication': result['replication'],
            'avg_turnstile_wait': np.mean(m.turnstile_waits) if m.turnstile_waits else 0,
            'max_turnstile_wait': np.max(m.turnstile_waits) if m.turnstile_waits else 0,
            'avg_vendor_wait': np.mean(m.vendor_waits) if m.vendor_waits else 0,
            'fans_completed': m.fans_completed,
            'fans_exited': getattr(m, 'fans_exited', 0),
        })
    
    results_df = pd.DataFrame(export_data)
    
    # Save to CSV
    export_path = Path('../data/processed/baseline_vs_ml_results.csv')
    results_df.to_csv(export_path, index=False)
    
    print('\n📤 EXPORT COMPLETE')
    print('=' * 80)
    print(f'✅ Results exported to: {export_path}')
    print(f'   Total records: {len(results_df)}')
    print(f'   Scenarios: {results_df["scenario"].nunique()}')
    print(f'   Replications per scenario: {NUM_REPLICATIONS}')
    print('=' * 80)
    
    # Show sample
    print('\n📋 Sample Data:')
    print(results_df.head(10))